# Tarea 3

Matias Correa\
15634183

In [ ]:
import math
from random import randint
from hashlib import md5 as _md5

### Algoritmo Extendido de Euclides e Inverso modular
Retorna (MCD(a,b), s, t) tal que MCD(a,b) = s\*a + t\*b

Obtenido de las [notas de clases](https://github.com/UC-IIC3253/2021/blob/main/notas%20de%20clases/05-27.pdf) y de [Wikipedia](https://en.wikipedia.org/wiki/Extended_Euclidean_algorithm)

In [ ]:
def alg_ext_euclides(a: int, b: int) -> (int, int,int):
    old_r, r = a, b
    old_s, s = 1, 0
    old_t, t = 0, 1

    while r:
        q = old_r // r
        old_r, r = r, old_r - q * r
        old_s, s = s, old_s - q * s
        old_t, t = t, old_t - q * t
    return old_r, old_s, old_t

def inverso(a: int, n: int) -> int:
    _, s, _ = alg_ext_euclides(a, n)
    while s < 0:
        s += n
    return s

### Exponenciación rápida con modulo
(a ** b) mod n\
Algoritmo basado en la respuesta a [esta pregunta de StackOverflow](https://stackoverflow.com/questions/40578553/fast-modular-exponentiation-help-me-find-the-mistake)

In [ ]:
def exp_mod(a: int, b: int, n: int) -> int:
    t = 1
    if b < 0:
        b = -b
        a = inverso(a, n)
    while b:
        t = (t * (a if b%2 else 1)) % n
        a = (a * a) % n
        b >>= 1
    return t

### ElGamal
Lee el grupo desde `grupo.txt`\
Las claves quedan almacenadas en `public_key.txt` y `private_key.txt`

In [ ]:
def generar_clave_ElGamal() -> None:
    with open('grupo.txt', 'r') as arch:
        p, g, q = (int("".join(i.strip().split(" ")), 16) for i in arch.readlines())
    x = randint(1, q-1)
    y = exp_mod(g, x, p)
    with open('private_key.txt', 'w') as arch:
        arch.write(str(x))
    with open('public_key.txt', 'w') as arch:
        arch.write(str(y))

### MD5

Wrapper de la implementacion de la libreria `hashlib`

In [ ]:
def md5(m: str) -> int:
    h = _md5()
    h.update(m.encode())
    return int(h.hexdigest(), 16)

### Firma de Schnorr

Recibe el mensaje `m` a firmar y entrega la firma `(e, s)`

In [ ]:
def firmar_Schnorr(m: str) -> (int, int):
    with open('grupo.txt', 'r') as arch:
        p, g, q = (int("".join(i.strip().split(" ")), 16) for i in arch.readlines())
    with open('private_key.txt', 'r') as arch:
        x = int(arch.readline().strip())
    
    k = randint(1, q-1)
    r = exp_mod(g, k, p)
    e = md5(str(r) + m)
    s = k - x*e
    return (e, s)

### Verificar firma de Schnorr

Retorna `True` si y solo si la firma `(e,s)` provista corresponde a una firma de Schnorr valida para el mensaje `m`.

In [ ]:
def verificar_firma_Schnorr(m: str, firma: (int, int)) -> bool:
    with open('grupo.txt', 'r') as arch:
        p, g, q = (int("".join(i.strip().split(" ")), 16) for i in arch.readlines())
    with open('public_key.txt', 'r') as arch:
        y = int(arch.readline().strip())

    e, s = firma
    r = (exp_mod(g, s, p) * exp_mod(y, e, p)) % p
    _e = md5(str(r) + m)

    return e == _e